## Select articles: look for hateful comments

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from mongoengine import connect
from hatespeech_models import Tweet, Article

client = connect("hatespeech-labelling")
db = client["hatespeech-labelling"]

In [2]:
Article.objects.count()

55380

In [3]:
db.article.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'hatespeech-labelling.article'},
 'comments.tweet_id_1': {'v': 2,
  'key': [('comments.tweet_id', 1)],
  'ns': 'hatespeech-labelling.article',
  'background': False},
 'created_at_1': {'v': 2,
  'key': [('created_at', 1)],
  'ns': 'hatespeech-labelling.article',
  'background': False},
 'tweet_idslug_1': {'v': 2,
  'key': [('tweet_idslug', 1)],
  'ns': 'hatespeech-labelling.article',
  'background': False},
 'user_1': {'v': 2,
  'key': [('user', 1)],
  'ns': 'hatespeech-labelling.article',
  'background': False},
 'body_text_title_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'hatespeech-labelling.article',
  'background': False,
  'default_language': 'spanish',
  'weights': SON([('body', 1), ('title', 1)]),
  'language_override': 'language',
  'textIndexVersion': 3},
 'slug_1': {'v': 2,
  'unique': True,
  'key': [('slug', 1)],
  'ns': 'hatespeech-labelling.article',
  'background': False,
  'sparse': False}}

In [52]:
ap = set()


Docstring:
Add an element to a set.

This has no effect if the element is already present.
Type:      builtin_function_or_method


In [ ]:
seeds = [    
    "China", "chino", "Cuba", "cubano", "bolivia",
    "Trump", "judío", "camionero", "agresor", 
    "policía", "ladrón", "reprimir", "represión",
    "juez", "justicia", "penal", "criminal", 
    "delito", "mamá", "género", 
    "aborto", "actriz", "feminista", "madre", "enfermera", "médica", 
    "piqueteros", "villa", "monos",
    "Agustina", "Pichot",
    "femicidios", "mujer", "trans",
    "cristina", "macri", "morales", "canosa", "evo",
    "\"movimientos sociales\"",
    "protesta",
    "\"organizaciones sociales\"",
    "\"villa\"",
    "\"Nati Jota\"",
    "\"Viale\"",
    "\"Ernestina Pais\"",
    "\"Mica Viciconte\"",
    "\"la modelo\"",
    "\"la actriz\"",
    "\"la periodista\"",
    "\"Carla\"",
    "\"la conductora\"",
    "\"la cantante\"",
    "\"Cande\"",
    "\"personal doméstico\"",
    "\"empleadas\"",
]

seed_ids = set()
seed_articles = []

queries = {}

for word in seeds:
    query = Article.objects(dummy__ne=True, comments__19__exists=True).search_text(word)
    
    nuevos = 0
    queries[word] = query
    
    for art in query:
        if art.id not in seed_ids:
            nuevos += 1
            seed_ids.add(art.id)
            seed_articles.append(art)
    print(f"{word} ---> {len(seed_articles)}, {nuevos} nuevos")
    
len(seed_articles)


China ---> 712, 712 nuevos
chino ---> 712, 0 nuevos
Cuba ---> 845, 133 nuevos
cubano ---> 886, 41 nuevos
bolivia ---> 974, 88 nuevos
Trump ---> 1101, 127 nuevos
judío ---> 1123, 22 nuevos
camionero ---> 1197, 74 nuevos
agresor ---> 1249, 52 nuevos
policía ---> 1829, 580 nuevos
ladrón ---> 1874, 45 nuevos
reprimir ---> 1905, 31 nuevos
represión ---> 1914, 9 nuevos
juez ---> 2202, 288 nuevos
justicia ---> 2708, 506 nuevos
penal ---> 2802, 94 nuevos
criminal ---> 2813, 11 nuevos
delito ---> 2875, 62 nuevos
mamá ---> 3004, 129 nuevos
género ---> 4081, 1077 nuevos
aborto ---> 4097, 16 nuevos
actriz ---> 4173, 76 nuevos
feminista ---> 4178, 5 nuevos
madre ---> 4307, 129 nuevos
enfermera ---> 4357, 50 nuevos
médica ---> 4717, 360 nuevos
piqueteros ---> 4728, 11 nuevos
villa ---> 4851, 123 nuevos
monos ---> 4862, 11 nuevos
Agustina ---> 4895, 33 nuevos
Pichot ---> 4895, 0 nuevos
femicidios ---> 4895, 0 nuevos
mujer ---> 4968, 73 nuevos
trans ---> 4972, 4 nuevos
cristina ---> 5145, 173 nuevos
m

## Cuál es la intersección con los que tienen odio?

In [115]:
initial_query = {
    "comments__19__exists": True,
    "dummy__ne": True,
}
articles = Article.objects(**initial_query)
articles = list(articles)

print(len(articles))


6519


In [116]:
for article in articles:
    hateful_comments = [c for c in article["comments"] if c["hateful_value"] > 0.5]
    
    article["num_hateful_comments"] = len(hateful_comments)
    article["avg_hateful_comments"] = len(hateful_comments) / len(article["comments"])
    article["avg_hate_value"] = sum(c["hateful_value"] for c in article["comments"]) / len(article["comments"])


In [117]:
threshold = 0.20


hateful_articles = [art for art in articles if art["avg_hateful_comments"] >= threshold]

percent = sum(1 for art in hateful_articles if art.id in seed_ids) / len(hateful_articles)

print(f"Para threshold {threshold:.2f}, tenemos {len(hateful_articles)} artículos")
print(f"De estos artículos, el {percent*100:.2f}% está en los que elegimos con seeds")

Para threshold 0.20, tenemos 1722 artículos
De estos artículos, el 84.73% está en los que elegimos con seeds


¿Cuáles no están?

In [119]:
not_found = [art for art in hateful_articles if art.id not in seed_ids]

len(not_found)

263

In [112]:
for i, art in enumerate(not_found):
    print((f"{'=' * 80 }" + '\n') * 3)
    print(f"{i+1} -- {art}")
    print("\n\n")
    print(art.body[:600])


1 -- 1244410425191682049 - clarincom (23 comentarios)
Coronavirus en Argentina: en 10 días hubo más de 33 mil detenidos por incumplir el aislamiento



Los controles se repiten y son cada vez más severos. Sólo entre el sábado a la noche y el domingo a las 15 controlaron a 41 mil personas y detuvieron a 1.396 por incumplir el aislamiento social, preventivo y obligatorio por el coronavirus.

Desde que empezó, el 20 de marzo, ya suman 13.510 -entre detenidos, demorados y notificados- sólo por fuerzas federales, a los que se acumulan más de 20.000 de las policías provinciales. Es decir, que en 10 días totalizaron más de 33.000.

En los operativos a cargo del Ministerio de Seguridad de la Nación se ha controlado a casi medio millón de personas en 

2 -- 1277043240865316864 - infobae (78 comentarios)
Amalia Granata criticó el endurecimiento del aislamiento: “No entiendo cómo la gente sana hace cuarentena”



Luego de 100 días de confinamiento y en un escenario de crisis económica y fatiga s

In [121]:
art = not_found[223]
art, art.comments

(1276621682703241217 - infobae (44 comentarios)
 La repudiable declaración de Bernie Ecclestone: “En muchos casos las personas negras son más racistas que las blancas”,
 [(0.99) @infobae Acertada,
  (0.03) @infobae https://t.co/i9B4aW9VMt,
  (0.01) @infobae ?????,
  (0.79) @infobae Y que tiene de desacertada?,
  (0.96) @infobae Todos los empleados afro americanos que contrate me dijeron EXACTAMENTE lo mismo. Es mas, me enseñaron como tienen MUCHO mas miedo morir en sus barrios por otra persona afro americana que por un blanco.,
  (0.00) @infobae Tiene razón.  No es políticamente correcto decirlo. Pero es cierto,
  (0.99) @infobae Bueno, en USA el "Black lives matter" no tiene sentido porque las estadísticas dicen que los negros matan más blancos, y la policía mata más blancos que negros. https://t.co/UKYTBoAVLn,
  (0.03) @infobae https://t.co/E78N0VS2Kw,
  (0.01) @infobae Es la verdad,
  (0.24) @infobae Tiene razón, el tema es que los negros tienen resentimiento contenido, bueno la F1 